In [ ]:
!pip install OpenAI
!pip install langchain --upgrade langchain
!pip install langchain-community langchain-core
!pip install chromadb
!pip install tiktoken
!pip install pypdf
!pip install langchain_openai
!pip install pandas
!pip install -qU langchain-community

import pandas as pd
from langchain_openai import ChatOpenAI
import tiktoken
import json
import torch
import torch.nn as nn
import numpy as np
import math
import torch.nn.functional as F
from langchain.llms import OpenAI
from torch import Tensor


In [ ]:
import torch
import torch.nn as nn
import tiktoken
import os
import json
import math
from langchain.tools.json.tool import JsonSpec
from langchain.agents.agent_toolkits import create_json_agent, JsonToolkit
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "your key"

encoding = tiktoken.encoding_for_model("gpt-4o")

class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        pe = torch.zeros(max_len, 1, embedding_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

def tea(prompt):
    tokenized_data = encoding.encode(prompt)
    vocab_size = encoding.n_vocab
    embedding_dim = 768

    embeddingfunction = nn.Embedding(vocab_size, embedding_dim)
    positional_encoding = PositionalEncoding(embedding_dim)
    multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=4, batch_first=True)
    linear_layer = nn.Linear(embedding_dim, vocab_size)

    optimizer = torch.optim.Adam(list(embeddingfunction.parameters()) +
                                 list(multihead_attn.parameters()) +
                                 list(linear_layer.parameters()), lr=1e-4)

    checkpoint_path = 'model_checkpoint.pth'

    def load_pretrained_model(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        embeddingfunction.load_state_dict(checkpoint['embeddingfunction_state_dict'])
        multihead_attn.load_state_dict(checkpoint['multihead_attn_state_dict'])
        linear_layer.load_state_dict(checkpoint['linear_layer_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded pre-trained model.")

    if os.path.exists(checkpoint_path):
        load_pretrained_model(checkpoint_path)
        training = False
    else:
        training = True

    if training:
        texts = [
            "Please keep the x coordinates between -10 to 10 and I would like one of the x-coordinates to be as close to 5 as possible.",
            "I would like to minimize the size of the building while still keeping it larger than 5 meters cubed.",
            "Please change the value of one of the x-values to be as close to 0 as possible.",
            "To optimize energy efficiency, adjust the HVAC settings to maintain a comfortable temperature while reducing power consumption during non-peak hours.",
            "Installing LED lighting and utilizing natural daylight can reduce electricity usage in commercial buildings.",
            "Enhancing insulation in the building envelope helps retain heat during the winter and keeps the building cool during the summer, leading to lower energy usage.",
            "Incorporating solar panels into the building's design can provide renewable energy, reducing dependency on grid electricity and lowering energy costs.",
            "An Energy Management System can monitor real-time energy usage and provide insights for optimizing energy efficiency across different building systems.",
        ]
        tokenized_texts = [encoding.encode(text) for text in texts]

        def train_model(tokenized_texts, epochs=100):
            for epoch in range(epochs):
                total_loss = 0
                for tokens in tokenized_texts:
                    tokens = torch.tensor(tokens).unsqueeze(0)
                    embeddings = embeddingfunction(tokens)
                    pos_encoded = positional_encoding(embeddings)
                    attn_output, _ = multihead_attn(pos_encoded, pos_encoded, pos_encoded)
                    logits = linear_layer(attn_output)

                    targets = tokens.squeeze(0)[1:]
                    logits = logits.squeeze(0)[:-1]

                    loss = nn.CrossEntropyLoss()(logits.view(-1, vocab_size), targets)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_loss += loss.item()
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss/len(tokenized_texts)}")

        train_model(tokenized_texts, epochs=100)

        torch.save({
            'embeddingfunction_state_dict': embeddingfunction.state_dict(),
            'multihead_attn_state_dict': multihead_attn.state_dict(),
            'linear_layer_state_dict': linear_layer.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, checkpoint_path)

    tokenized_tensor = torch.tensor(tokenized_data, dtype=torch.long).unsqueeze(0)
    embedding = embeddingfunction(tokenized_tensor)
    encoded_tokens = positional_encoding(embedding)
    attn_output, _ = multihead_attn(encoded_tokens, encoded_tokens, encoded_tokens)
    logits = linear_layer(attn_output)
    predicted_token_ids = torch.argmax(logits, dim=-1)
    return encoding.decode(predicted_token_ids.squeeze().tolist())

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o")

uploaded_file = "1ZoneUncontrolled.epJSON"

with open(uploaded_file, "r") as file:
    content = file.read()

data = json.loads(content)
spec = JsonSpec(dict_=data)
toolkit = JsonToolkit(spec=spec)
agent_executor = create_json_agent(llm, toolkit=toolkit, verbose=True)

parameters = tea("Change the height of the building to 5 meters")

output = agent_executor.run(f"You are given an epJSON file of a building with various different details and parameters. We would like you to do {parameters}.")

print(parameters)


In [ ]:
file_idf = '%s_%s.idf'%(file_idf_org[:-4],i)
            command = '%s\\energyplus.exe -r ' %path_EP + \
                    '-d "%s\\%s" ' %(path_output,file_idf[:-4]) + \
                    '-w "%s" '%(file_weather) + \
                    '-p "%s" '%(file_idf[:-4]) + \
                    '"%s\\%s\\%s"' %(path_input, file_idf[:-4], file_idf)
            print(command)

            slots['process'][idx] = subprocess.Popen(command)